In [1]:
# Import das bibliotecas necessárias

import pandas as pd

from google.colab import drive

import nltk
import re
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE

# TODO: Incluir o enelvo ao projeto de forma otimizada para normalizar textos
#!pip install enelvo
#from enelvo.normaliser import Normaliser


In [2]:
# Download dos pacotes úteis do NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Conecta ao Google Drive para trabalhar com arquivos em nuvem
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Lê e armazena os dados de treinamento
train_data = pd.read_csv('/content/drive/MyDrive/I2A2-2023-Sentiment_Analysis/train.csv', index_col='ID', usecols=['ID', 'review_title', 'review_text', 'recommend_to_a_friend', 'rating'])

train_data.head()

,review_title,review_text,recommend_to_a_friend,rating
ID,,,,
1,Bom,Estou contente com a compra entrega rápida o ú...,Yes,4
2,"Preço imbatível, ótima qualidade","Por apenas R$1994.20,eu consegui comprar esse ...",Yes,4
3,ATENDE TODAS AS EXPECTATIVA.,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,Yes,4
4,Produto imperdível,"Excelente produto, por fora em material acríli...",Yes,5
5,ótimo,"produto mto bom, com essa garrafinha vc pode a...",Yes,5


In [5]:
# Lê e armazena os dados de teste do modelo
test_data = pd.read_csv('/content/drive/MyDrive/I2A2-2023-Sentiment_Analysis/test.csv', index_col='ID', usecols=['ID', 'review_title', 'review_text', 'recommend_to_a_friend'])

test_data.head()

,review_title,review_text,recommend_to_a_friend
ID,,,
79426,Excelente,"Celular muito bom, a imagem dele em jogos impr...",Yes
79427,Excelente custo x benefício,Excelente produto! Vale a pena o investimento!...,Yes
79428,Produto exelente,"Fiquei muito satisfeita com minha compra, cheg...",Yes
79429,Gostei muito do produto,Sem comentarios..................................,Yes
79430,"Ótimo produto, entrega rápida e excelente!!!","Sofá grande, confortável. Entrega muito antes...",Yes


In [6]:
# Define função para limpeza dos textos
def text_cleaning(text):
    # Substitui links e normaliza os textos retirando pontuações e transformando as palavras em minúsculas
    text = " ".join(re.findall(r'\b[A-zÀ-úü]+\b', text))
    text = re.sub(r"http\S+||\S+\.com\S{0,}", "", text.lower()).translate(str.maketrans('', '', string.punctuation))

    # TODO: Otimizar o processo de utilização do enelvo para remover abreviações
    #normaliser = Normaliser(tokenizer='readable', sanitize=True)
    #text = normaliser.normalise(text)


    # Retira stopwords da string e transforma todas as palavras em seus infinitivos
    lemmatizer = nltk.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [lemmatizer.lemmatize(i) for i in text.split() if not i in stopwords]

    # Retorna os textos limpos
    return (" ".join(words))

In [7]:
# Define função para pré-processar todos os textos de um dataset
def pre_process(data):
    # Substitui todos os dados NaN por strings vazias
    data = data.fillna('')

    # Cria uma nova coluna no dataset concatenando o texto e título da review com o campo de recomendação a um amigo
    data['review'] = data['review_title'] + ' ' + data['review_text'] + ' ' + data['recommend_to_a_friend']

    # Executa a limpeza dos textos a todas as linhas da coluna de resumo da review
    reviews_processed = [text_cleaning(t) for t in data['review']]

    # Retorna as reviews pré-processadas
    return reviews_processed

In [8]:
# Define o eixo Y (target) de treino como os valores da coluna de notas do dataset de treino
y_train = train_data['rating']

# Utiliza a ferramente de vetorização de textos da scikit-learn
vetorizer = CountVectorizer(analyzer="word")

# Define o eixo X (feature) de treino vetorizando os textos pré-processados do dataset de treino
x_train = vetorizer.fit_transform(pre_process(train_data))

# Define o eixo X (feature) de treino vetorizando os textos pré-processados do dataset de teste baseando nos vetores de treino
x_test = vetorizer.transform(pre_process(test_data))

In [9]:
# TODO: Melhorar resultados do Smote ao utilizar dados artificiais para balancear os dados de treino
#smote = SMOTE()
#smote_x, smote_y = smote.fit_resample(x_train, y_train)

# Define o modelo baseado no algoritmo Multinomial Naive Bayes
model = MultinomialNB(alpha=1.0, force_alpha='warn', fit_prior=True, class_prior=None)

# Treina o modelo utilizando os dados de treino pré-processados
model.fit(x_train, y_train)

MultinomialNB()

In [10]:
# Executa a predição dos dados de teste utilizando o modelo treinado
y_test = model.predict(x_test)

# Adiciona os dados preditos ao dataset de teste com uma nova coluna de notas
test_data['rating'] = y_test

# Cria uma cópia do dataset de teste usando apenas as colunas de ID e rating
submission = test_data[['rating']]

# Salva os dados de submissão como csv na pasta do Google Drive
submission.to_csv('/content/drive/MyDrive/I2A2-2023-Sentiment_Analysis/final_submission.csv')

submission.head()

,rating
ID,
79426,5
79427,5
79428,5
79429,5
79430,5
